In [1]:

import pandas as pd
file_path = r'C:\Users\PRAVEEN CHANDRA\Desktop\Credit_Banking_2.xlsx'
Worksheet = pd.ExcelFile(file_path)

# Get the list of sheet names in the Excel file
Worksheetname = Worksheet.sheet_names

# Access each table individually
Credit_transaction = pd.read_excel(Worksheet, sheet_name=Worksheetname[0])
Customer_info = pd.read_excel(Worksheet, sheet_name=Worksheetname[1])

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
#filling null or NaN values in credit card column with 99999 
Credit_transaction['Credit_card'].fillna(99999, inplace=True)


In [ ]:
Credit_transaction.sort_values(by=['Credit_card','Time'],ascending=True,inplace=True)
duplicate_transactions = Credit_transaction[Credit_transaction.duplicated(subset=['Credit_card', 'Transaction ID'], keep=False)]
duplicate_transactions

In [25]:
#age>18
Customer_info.loc[Customer_info['Age']<18,'Age']=Customer_info.loc[Customer_info['Age']>=18,'Age'].mean()


In [27]:
Credit_transaction.rename(columns={'Date':'Purchase_date'},inplace=True)


In [ ]:
#check return_date is after purchase_date or not
Credit_transaction.loc[Credit_transaction['Purchase_date']>Credit_transaction['Return_date']]

In [ ]:
Credit_transaction.loc[Credit_transaction['Purchase_date']>Credit_transaction['Return_date'],'Return_date']=Credit_transaction['Return_date'] + pd.DateOffset(years=1)


In [43]:
Credit_transaction.loc[Credit_transaction['Coupon_ID'].isnull(),'Selling_price']=Credit_transaction['Price ']

In [42]:
condition = (Credit_transaction['Price '] == Credit_transaction['Selling_price']) & (~Credit_transaction['Coupon_ID'].isnull())
Credit_transaction.loc[condition, 'Selling_price'] *= 0.95

In [ ]:
Combined=pd.merge(Credit_transaction,Customer_info,left_on='Credit_card',right_on='C_ID')
Combined

In [ ]:
age_bins = [18, 36, 51, 100]
age_labels = ['Young', 'Mid age', 'Old']

# Create a new column 'age_group' based on the age bins and labels
Combined['age_group'] = pd.cut(Combined['Age'], bins=age_bins, labels=age_labels, right=False)
Combined['age_group']=Combined['age_group'].astype(str)
# Create a new column 'segment' combining age group and gender
Combined['segment'] = Combined['age_group'] + ' ' + Combined['Gender']
Combined

In [ ]:
spend_segmentation=Combined.groupby('segment')['Selling_price'].sum()
spend_segmentation

In [ ]:
Swipe_segmentation=Combined.groupby('segment')['Credit_card'].value_counts()
Swipe_segmentation

In [ ]:
spend_by_product=Combined.groupby('P_CATEGORY')['Selling_price'].sum()
spend_by_product

In [ ]:
spend_by_state=Combined.groupby('State')['Selling_price'].sum()
spend_by_state

In [ ]:
spend_by_payment_method=Combined.groupby('Payment Method')['Selling_price'].sum()
spend_by_payment_method

In [ ]:
highest_paying_5_spend_by_product=spend_by_product.nlargest(5)
highest_paying_5_spend_by_product


In [ ]:
highest_paying_5_spend_by_state=spend_by_state.nlargest(5)
highest_paying_5_spend_by_state


In [ ]:
highest_paying_5_spend_by_payment_method=spend_by_payment_method.nlargest(5)
highest_paying_5_spend_by_payment_method

In [ ]:
Combined['Discount']=Combined['Price ']-Combined['Selling_price']
payment_met_highest_discount=Combined.groupby('Payment Method')['Discount'].mean()
payment_met_highest_discount.sort_values(ascending=False)

In [ ]:
returned_customers=Combined[Combined['Return_ind']==1]

In [ ]:
age_category=returned_customers.groupby('age_group').size()
age_category

In [ ]:
returned_customers.groupby('CONDTION').size()

In [ ]:
returned_customers.groupby('State').size()

In [ ]:
returned_customers.groupby('Discount').size()

In [ ]:
discount_bins=[0,50,100,150,200,250]
discount_labels=['0-50','51-100','101-150','151-200','201-250']
returned_customers['discount_group'] = pd.cut(returned_customers['Discount'], bins=discount_bins, labels=discount_labels, right=False)

returned_customers.groupby('discount_group').size()

In [92]:
non_returning_customers=Combined[Combined['Return_ind']==0]


In [ ]:
discount_bins=[0,50,100,150,200,250]
discount_labels=['0-50','51-100','101-150','151-200','201-250']
non_returning_customers['discount_group'] = pd.cut(non_returning_customers['Discount'], bins=discount_bins, labels=discount_labels, right=False)

non_returning_customers.groupby('discount_group').size()
